In [17]:
import pyspark
import pandas as pd
df = pd.DataFrame([2,3,4])
df

,0
0,2
1,3
2,4


In [2]:
sc = pyspark.SparkContext()

In [12]:
#con esto podemos trabajar con los RDD cutres
sc.parallelize([1,2,3]).count()

3

In [16]:
# si creamos un contexto de SQL (a partir del SparkContext)
# entonces ya podemos usar todas las funcionalidades de los dataframe de SQL
sql = pyspark.SQLContext(sc)

In [19]:
sql.createDataFrame(df).show(4)

+---+
|  0|
+---+
|  2|
|  3|
|  4|
+---+

